In [45]:
import sys
import os
from tqdm import tqdm
import numpy as np
import docplex.cp.utils_visu as visu
from docplex.cp.model import *
import collections
from io import StringIO
from itertools import permutations
import math
import copy 
import chart_studio.plotly as py
import plotly.express as px
import pandas as pd
from datetime import datetime
# import tensorflow as tf
from itertools import chain

In [46]:
NB_EXAMPLES = 100
PT_MEAN = 32
PT_STDDEV = 8
NB_JOBS_RANGE = [20, 20]
NB_MACHINE_RANGE = [4, 4]
NB_DATA = 1
NB_SETUP_TYPES = 5
JOB_INFLATION_FACTOR = 10
MACHINE_INFLATION_FACTOR = 10
NB_TEST_SAMPLES = 10

In [47]:
def print_full_df(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [48]:
def solve(proc_t,due_dates,rel_dates,nb_m,setup_types,return_model,return_vars,return_obj=False):
    """
    REF: https://ibmdecisionoptimization.github.io/tutorials/html/Scheduling_Tutorial.html
    http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.model.py.html

    Inspired by House Building Problem, hence the following terms are defined:

    Worker => Machines
    Tasks => Jobs
    Houses => 1 (does not fulfill a purpose here)
    Skills => each machine has the skill to process each job
    Deadline => a day
    """
    NbHouses = 1
    Deadline =  24*60

    Workers = ["M"+str(i) for i in range( nb_m)]

    Tasks = ["T"+str(i) for i in range( proc_t.shape[0])]

    Durations = proc_t
    ReleaseDate = rel_dates
    DueDate = due_dates

    Skills=[]
    for w in Workers:
        for t in Tasks:
            Skills.append((w,t,1))

    nbWorkers = len(Workers)
    Houses = range(NbHouses)
    mdl5 = CpoModel()
    tasks = {}
    wtasks = {}
    wseq = {}
    transitionTimes = transition_matrix(len(Tasks))
    for h in Houses:
        for i,t in enumerate(Tasks):
            # add interval decision var for each job, range from 0 to Deadline, and fixed length of PT
            # thus each task has to fit with its pt in the fictional deadline (max time)
            tasks[(h,t)] = mdl5.interval_var(start=[0,Deadline], size=Durations[i])

            # Add transition times between tasks, which do NOT share the same setup time
            for j,t2 in enumerate(Tasks):
                if np.dot(setup_types[i], setup_types[j]) == 0:
                    transitionTimes.set_value(i, j, 10)
                else:
                    transitionTimes.set_value(i, j, 0)

        for i,s in enumerate(Skills):
            # looping over each possible combi of machine and job (skill)
            # add interval decision var for each combi, range from 0 to DD for each job.
            # Thus each job on each machine must be processed within a range of 0 upto its DD.
            # this is optional, thus do not have to be fulfilled?
            wtasks[(h,s)] = mdl5.interval_var(start=[0,DueDate[i%len(Tasks)]],optional=True)
        for w in Workers:
            wseq[w] = mdl5.sequence_var([wtasks[(h,s)] for s in Skills if s[0] == w],
                                        types=[int(s[1][1:]) for s in Skills if s[0] == w ])
    for h in Houses:
        for t in Tasks:
            # add constraint such that if j is in the solution space, then there is exactly one job on a machine.
            mdl5.add( mdl5.alternative(tasks[h,t], [wtasks[h,s] for s in Skills if s[1]==t]) )
    for w in Workers:
        
        # add overlap constraint to enforce transitions is required
        mdl5.add( mdl5.no_overlap(wseq[w], transitionTimes))

    # length_of gives us the time end-start of a job, hence we minimize the time a job has to wait
    # with is the dual problem to maximizing the time gap to its dd
    mdl5.add(
        mdl5.maximize(
            mdl5.sum(mdl5.size_of(wtasks[h,s]) - mdl5.end_of(wtasks[h,s])
                     for h in Houses for s in Skills)
        )
    )

    # Solve the model
    solver_log_stream = StringIO()
    msol5 = mdl5.solve(log_output=solver_log_stream)

    # transform model solution to a format, which can be handled afterwards
    worker_idx = {w : i for i,w in enumerate(Workers)}
    worker_tasks = [[] for w in range(nbWorkers)]  # Tasks assigned to a given worker
    for h in Houses:
        for s in Skills:
            worker = s[0]
            wt = wtasks[(h,s)]
            worker_tasks[worker_idx[worker]].append(wt)
    sol_dict = {k: [] for k in range(nb_m)}

    for i,w in enumerate(Workers):
        visu.sequence(name=w)
        for k,t in enumerate(worker_tasks[worker_idx[w]]):
            wt = msol5.get_var_solution(t)
            #print(wt)
            if wt.is_present():
                sol_dict[i].append((k,wt.start,wt.end))
    for i,w in enumerate(Workers):
        sol_dict[i].sort(key=lambda tup: tup[1])

    return_list = [sol_dict]
    if return_obj:
        return_list.append(msol5.get_objective_values()[0])
    if return_model:
        return_list.append(msol5)
    if return_vars:
        return_list.append(transitionTimes)
    return return_list

In [49]:
def gen_due_date(max_proc_t,nb_m,nb_t,pt_int=True):
    proc_l=[]
    fac_vec=np.arange(np.ceil(nb_t/2),dtype=np.int64)+1
    fac_vec=np.repeat(fac_vec, 2)
    print(fac_vec)
    np.random.shuffle(fac_vec)

    for i in range(nb_t):
        t=np.random.normal(loc=(fac_vec[i]*max_proc_t[i]+max_proc_t[i]/2),scale=max_proc_t[i]/6)
        if pt_int:
            proc_l.append(int(t))
        else:
            proc_l.append(t)
    return np.array(proc_l)

In [50]:
NB_MACHINE_RANGE = [3, 3]
NB_JOB_RANGE = [3, 3]
PT_MEAN = 32
PT_STDDEV = 8
nb_m = np.random.randint(NB_MACHINE_RANGE[0],NB_MACHINE_RANGE[1]+1) if type(NB_MACHINE_RANGE) is list else NB_MACHINE_RANGE
nb_t = np.random.randint(NB_JOB_RANGE[0],NB_JOB_RANGE[1]+1) if type(NB_JOB_RANGE) is list else NB_JOB_RANGE
proc_t=np.random.normal(loc=PT_MEAN,scale=PT_STDDEV,size=(NB_JOB_RANGE))
proc_t=proc_t.astype(np.int64)
d = gen_due_date(proc_t,nb_m,nb_t,pt_int=False)
# print(proc_t)
# print(d)

[1 1 2 2]


In [51]:
def gen_setup_types(nb_t, nb_s):
    # nb_s is the intended number of different setup tasks
    setup_types = [np.random.randint(0, nb_s - 1) for _ in range(nb_t)]
    setup_types = np.array(setup_types).reshape(-1)
    setup_types = np.eye(nb_s)[setup_types]
    return setup_types

In [52]:
def get_pt(mean, stddev=0.0, size=1):
    if stddev: 
        pt = np.random.normal(loc=mean,scale=stddev,size=size)
        return pt.astype(np.int64)
    return np.fill(mean, size, np.int64)

In [53]:
def simulation(pt_mean,
               pt_std,
               nb_t_range,
               nb_m_range,
               nb_data,
               nb_s,
               pt_int=False,
               return_dur=False,
               return_pts=False):
    # init parameters
    # pt_mean: mean of processtime (duration of task on every machine)
    # pt_std:std of processtime
    # number of task range f.e. [2,10] means uniform distribution between 2 and 10 tasks
    # number of machine range f.e. [2,10] means uniform distribution between 2 and 10 machines
    # nb_data = number of datasamples which should get generated
    # pt_int = (boolean) if process time is int or float (default: False)
    # nb_s = number of setup types
    # return_dur = process time get's return (for statistics, default:False)
    x=[]
    y=[]
    if return_dur:
        dur=[]
    for i in range(nb_data):

        nb_m = np.random.randint(nb_m_range[0],nb_m_range[1]+1) if type(nb_m_range) is list else nb_m_range
        nb_t = np.random.randint(nb_t_range[0],nb_t_range[1]+1) if type(nb_t_range) is list else nb_t_range

        if pt_int:
            #proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_m,nb_t))
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_t))
            proc_t=proc_t.astype(np.int64)
        else:
            #proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_m,nb_t))
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=(nb_t))

        rel_dates = np.zeros(nb_t,dtype=np.int32)
        #due_dates=gen_due_date(np.amax(proc_t, axis=0),nb_m,nb_t)
        due_dates = gen_due_date(proc_t,nb_m,nb_t)
        
        # ENABLE SETUP TYPES
        if True:
            setup_types = gen_setup_types(nb_t, nb_s)
        else:
            setup_types = np.ones([nb_t, nb_s])

        x.append([{
            'pt':proc_t,
            'dd':due_dates,
            'st':setup_types
        }])

        sol=solve(proc_t, due_dates, rel_dates, nb_m, setup_types,
                  return_model=True,return_vars=True)
        sol, model, transitionTimes = sol[:-2], sol[-2], sol[-1]
        y.append([sol])
        if return_dur:
            dur.append(sol[-1])
    if return_dur:
        return x,y,dur
    if return_pts:
        return x,y,proc_t,model,transitionTimes
    return x,y

In [54]:
total_data = {'x': [], 'y': []}
for i in tqdm(range(NB_EXAMPLES)):
    try:
        x,y = simulation(PT_MEAN,PT_STDDEV,NB_JOBS_RANGE,
                         NB_MACHINE_RANGE,NB_DATA,NB_SETUP_TYPES,True)
        total_data.get('x').extend(x[0])
        total_data.get('y').extend(y[0][0])
    except:
        pass
print('done')

 27%|█████████████████████████▉                                                                      | 27/100 [00:00<00:00, 262.17it/s]

[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 271.43it/s]

[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10]
[ 1  1  

In [55]:
inf_total_data = {
    'x': total_data.get('x'),
    'y': total_data.get('y')
}

In [56]:
data_x = []
for example in range(len(inf_total_data.get('x'))):
    data_x_point = []
    point = inf_total_data.get('x')[example]
    for i in range(len(point.get('pt'))):
        t = [point.get('pt')[i], point.get('dd')[i]]
        t.extend(point.get('st')[i])
        data_x_point.append(t)
    data_x.append(data_x_point)
data_x = np.array(data_x, np.object)
print(data_x.shape)

(0,)


In [57]:
data_y = []
for example in range(len(inf_total_data.get('y'))):
    point = inf_total_data.get('y')[example]
    t = [point[m] for m in range(len(point.keys()))]
    data_y.append(t)
data_y = np.array(data_y, np.object)
print(data_y.shape)

(0,)


In [58]:
# padding input to get equal input job number
def pad_jobs(x):
    job_pad = np.append(np.array([-1, -1]), np.zeros(NB_SETUP_TYPES))
    padded_x = np.empty([len(list(x)), max(NB_JOBS_RANGE), len(job_pad)])
    padded_x[:] = job_pad
    for i in range(len(x)):
        padded_x[i, :len(x[i]), :] = x[i]
    return padded_x

# padding output to get equal machine number
# and jobs assigned to it
def pad_machines(y):
    job_pad = np.array([-1, -1, -1], np.float32)
    y_padded = np.zeros([len(y), max(NB_MACHINE_RANGE), max(NB_JOBS_RANGE), len(job_pad)])
    y_padded[:] = job_pad
    for i in range(len(y)):
        for m in range(len(y[i])):
            if len(y[i][m]) > 0:
                y_padded[i, m, :len(y[i][m]), :] = y[i][m]
    return y_padded

In [59]:
data_x = pad_jobs(data_x)
data_y = pad_machines(data_y)
print(data_x.shape)
print(data_y.shape)

(0, 20, 7)
(0, 3, 20, 3)


In [60]:
# add ID to jobs
data_x_copy = np.ones([data_x.shape[0], data_x.shape[1], data_x.shape[2]+1])
for i in range(len(data_x)):
    for j in range(len(data_x[i])):
        data_x_copy[i, j] = np.insert(data_x[i, j], 0, j)
data_x = data_x_copy
print(data_x.shape)

(0, 20, 8)


In [61]:
# numerical order for y to cut down the solution space
# sort by the first job id on each machine
# since when we consider an iterative solving alg, this sort style can be learned 
# even iteratively
data_y_copy = np.copy(data_y)
# print(data_y[0])
for i in range(len(data_y)):
    minima = []
    for m in range(len(data_y[i])):
        arr = data_y_copy[i,m,0,0]
        minimum = np.where(arr >= 0, arr, np.inf).min()
        minima.append(minimum)
    m_perm = np.argsort(minima)
    for m in range(len(data_y[i])):
        data_y[i, m] = data_y_copy[i, m_perm[m]]
# print(data_y[0]) 
        

In [62]:
def job_swap_inflation(samples: np.array, inflation_factor=10):
    inflated_samples = []
    perm_keys = []
    for i, sample in enumerate(samples):
            
        # randomly sample from the permutation set
        rnd_perms_keys = [np.random.permutation(len(sample)) for _ in range(inflation_factor)]
        perm_keys.append(rnd_perms_keys)
        
        # add combis to the inflation list
        for perm_key in rnd_perms_keys:
            inflated_sample = np.empty(sample.shape)
            inflated_sample[:, 0] = np.array([sample[i, 0] for i in perm_key])
            inflated_sample[:, 1] = np.array([sample[i, 1] for i in perm_key])
            inflated_sample[:, 2] = np.array([sample[i, 2] for i in perm_key])
            inflated_sample[:, 1:] = np.array([sample[i, 1:] for i in perm_key])
            inflated_samples.append(inflated_sample)
            
    return np.array(inflated_samples), np.squeeze(perm_keys)


def machine_swap_inflation(labels: np.array) -> np.array:
    inflated_labels = []
    for i, label in enumerate(labels):
            
            machine_alloc = label
            perms = permutations(np.arange(max(NB_MACHINE_RANGE)))
            for perm in perms:
                inflated_label = [machine_alloc[k] for k in perm]
                inflated_labels.append(inflated_label)
            
    return np.array(inflated_labels)


def inflate_ds(data, job_inflation_factor=10, machine_inflation_factor=10):
    inf_data = ([], [])
    data_x, data_y = data
    for x, y in tqdm(zip(data_x, data_y)):
        
        # inflate x and y
        x_job_inf, perm_keys = job_swap_inflation([x], inflation_factor=job_inflation_factor)
        # y_mas_inf = machine_swap_inflation([y])
        
        # assign random ys sampled from the inflated set to random jobs of the inflated set
        # Note that under the assumption that the previous inflation is correct, 
        # switching the high level ordering of the inflated samples and labels, does
        # NOT lead to infeasible examples, since all combinations are feasible anyway
#         y_idxs = np.random.randint(len(y_mas_inf), size=machine_inflation_factor)
        
        # when the job list is inflated after padding, the ids in the machine labels are also shuffled
        # to encounter for this, we change only the indexes in the machine label lists 
#         inf_data_y = [y_mas_inf[i] for i in y_idxs]

#         inf_data_y = np.array(inf_data_y)
#         for i in range(len(inf_data_y)):
#             for m in range(len(inf_data_y[i])):
#                 for j in range(len(inf_data_y[i][m])):
#                     if inf_data_y[i, m, j, 0] != -1:
#                         old = inf_data_y[i, m, j, 0]
#                         inf_data_y[i, m, j, 0] = list(perm_keys[i]).index(old)

        inf_data[0].extend(x_job_inf)
        inf_data[1].extend([y for _ in range(job_inflation_factor)])
        
    return inf_data

In [63]:
# (inf_data_x, inf_data_y) = inflate_ds((data_x, data_y),
#                                       job_inflation_factor=JOB_INFLATION_FACTOR,
#                                       machine_inflation_factor=MACHINE_INFLATION_FACTOR)
# inf_data_x = np.array(inf_data_x)
# inf_data_x = np.concatenate((inf_data_x, data_x))
# inf_data_y = np.array(inf_data_y)
# inf_data_y = np.concatenate((inf_data_y, data_y))
# print(inf_data_x.shape)
# print(inf_data_y.shape)
# print(inf_data_x[0])
# print(inf_data_y[0])
inf_data_x = data_x
inf_data_y = data_y

In [64]:
def y_valid_ms(y):
    return sum(y[:, 0, 0] != -1)
    
def check_point_validity(x, y):
#     id = x[:, 0] 
    pt = x[:, 0]    
    dd = x[:, 1]
    st = x[:, 2:]
    rd = np.zeros(len(pt), dtype=np.int32)
    nb_m = y_valid_ms(y)
    
    # remove padding 
    pt = pt[pt != -1].astype(np.int32)
    dd = dd[dd != -1].astype(np.int32)
    st = st[np.sum(st, axis=1) != 0, :]
    
    y_hat = solve(pt,dd,rd,nb_m,st,False,False,False)

    y_makespan = np.amax(y[:, :, 1])
    
    rd_y_hat = [np.array(alloc)[:, 2] for alloc in list(y_hat[0].values())]
    y_hat_makespan = max(list(chain(*chain(rd_y_hat))))
    
    return {'y': y_makespan, 'y_solver': float(y_hat_makespan), 'valid': y_makespan <= y_hat_makespan}

def random_validity_check(ds, nb_samples=10):
    ds_x, ds_y = ds
    rand_sample_idxs = np.random.randint(len(ds_x), size=nb_samples)
    results = [check_point_validity(ds_x[idx], ds_y[idx]) for idx in tqdm(rand_sample_idxs)]
    df = pd.DataFrame(results)
    return df

In [66]:
df = random_validity_check((inf_data_x, inf_data_y), nb_samples=5)
display(df) 

ValueError: low >= high

In [67]:
y_ele = inf_data_y[0]
df = pd.DataFrame([
    dict(Task="Job "+str(i),
         Start=str(datetime(2020, 1, 1, int(y_ele[m][i][1]//60), int(y_ele[m][i][1]%60))),
         Finish=str(datetime(2020, 1, 1, int(y_ele[m][i][2]//60), int(y_ele[m][i][2]%60))),
         Resource=str(m)
        ) for m in range(len(y_ele)) for i in range(len(y_ele[m])) if y_ele[m][i][0] != -1 
])
print_full_df(df)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
fig.show()
print(inf_data_x[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [68]:
# one hot encoding of labels y
def one_hot_encoder(y):
    M = max(NB_MACHINE_RANGE)
    J = max(NB_JOBS_RANGE)
    y_data_one_hot = np.zeros([len(y), M, J, J])
    for i in tqdm(range(len(y))):
        for m in range(M):
            for j in range(J):
                one_hot = np.zeros(J)
                idx = int(y[i, m, j, 0])
                one_hot[idx] = 1 if idx != -1 else 0
                y_data_one_hot[i, m, j] = one_hot
    y = np.transpose(y_data_one_hot, [0, 1, 3, 2])
    return y

In [69]:
pad_inf_data_x = inf_data_x
pad_inf_data_y = one_hot_encoder(inf_data_y)
print(pad_inf_data_x.shape)
print(pad_inf_data_y.shape)
# print(inf_data_y[20])
# print(pad_inf_data_y[20])

0it [00:00, ?it/s]

(0, 20, 8)
(0, 3, 20, 20)


In [70]:
df = pd.DataFrame({'j'+str(j): x[j] for j in range(len(x))} for x in pad_inf_data_x)
df.to_pickle('x.data')

In [71]:
df = pd.DataFrame({
    'm'+str(m): {'j'+str(j): y[m][j] for j in range(len(y[m]))}
for m in range(len(y))} for y in pad_inf_data_y)
df.to_pickle('y.data')

In [72]:
x_path = 'x.data'
y_path = 'y.data'

# x_path = '../../../../learner/res/ds/processed/setup_types/fixed_4m12j_setup_no_augm/x_large.data'
# y_path = '../../../../learner/res/ds/processed/setup_types/fixed_4m12j_setup_no_augm/y_large.data'

In [73]:
df = pd.DataFrame(pd.read_pickle(x_path))
x = []
for example in range(df.shape[0]):
    x.append([df['j'+str(job)][example] for job in range(df.shape[1])])
x = tf.convert_to_tensor(x, tf.float32)
print(x.shape)

NameError: name 'tf' is not defined

In [74]:
df = pd.DataFrame(pd.read_pickle(y_path))
y = []
for example in range(df.shape[0]):
    y_m = []
    for m in range(df.shape[1]):
        j_length = len(df['m'+str(m)][example].keys())
        y_m.append([df['m'+str(m)][example]['j'+str(j)] for j in range(j_length)])
    y.append(y_m)
y = tf.convert_to_tensor(y, tf.float32)
print(y.shape)

NameError: name 'tf' is not defined

In [75]:
import plotly.graph_objects as go

def sparse_to_dense(x, y, padding_value=-1):
    y = y.numpy()
    x = x.numpy()
    M = tf.shape(y)[1]
    T = tf.shape(y)[-2]
    B = tf.shape(y)[0]
    y_dense = np.empty([B, M, T, 7])
    for i in range(len(y)):
        for m in range(M):
            y_dense[i, m, :] = np.matmul(y[i, m].T, x[i, :, :])
            for t in range(T):
                if y_dense[i, m, t, 0] == 0:
                    y_dense[i, m, t, :2] = (padding_value, padding_value)
    return y_dense.astype(np.float32)

y_dense = sparse_to_dense(x, y)

AttributeError: 'list' object has no attribute 'numpy'

In [76]:
class_counts = y_dense[:, :, :, 0]
unique, counts = numpy.unique(class_counts, return_counts=True)
class_counts = dict(zip(unique, counts))
class_counts.pop(-1.0)

pt_distr = y_dense[:, :, :, 0].flatten()
pt_distr = pt_distr[pt_distr != -1]

dd_distr = y_dense[:, :, :, 1].flatten()
dd_distr = dd_distr[dd_distr != -1]
fig = go.Figure()

fig.add_trace(go.Histogram(x=pt_distr, histnorm='probability', name='pt'))
fig.add_trace(go.Histogram(x=dd_distr, histnorm='probability', name='dd'))
fig.update_layout(title="PT/DD Distr.", barmode='overlay')
fig.update_traces(opacity=0.75)

fig.show()

NameError: name 'y_dense' is not defined

In [77]:
# Not needed!
y_idx = np.empty([y.shape[0], y.shape[1], y.shape[2]])
for i in range(len(y)):
    for m in range(len(y[i])):
        for j in range(len(y[i,m])):
            y_idx[i,m,j] = -1 if np.sum(y[i,m,j]) == 0 else np.where(y[i,m,j] == 1)[0][0]

# i x m x j -> int means time pos
print(y_idx[0])

AttributeError: 'list' object has no attribute 'shape'

In [78]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 100em; }</style>"))

for i in range(y.shape[1]):
    fig = go.Figure(data=go.Heatmap(
            z=np.sum(y[:, i, :, :], axis=0),
            x=np.arange(y.shape[2]),
            y=np.arange(y.shape[2]),
            colorscale='Viridis'))

    fig.update_layout(title='Class Distr. M{}'.format(i), height=300)
    fig.show()

/tmp/ipykernel_2734/1428540441.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


AttributeError: 'list' object has no attribute 'shape'